In [1]:
from geopy.distance import distance
import geopandas as gpd
import datetime
import numpy as np

file = '/home/conor/Projects/D4D/data/connor_2020-02-23T16_50_40-07_00.geojson'

In [2]:
df = gpd.read_file(file)

def f_lat(x):
    b = x.bounds
    return b[1]

def f_lon(x):
    b = x.bounds
    return b[0]

def f_coords(x):
    b = x.bounds
    return (b[1], b[0])

df['lattitude'] = df['geometry'].apply(f_lat)
df['longitude'] = df['geometry'].apply(f_lon)
df['coordinates'] = df['geometry'].apply(f_coords)

df["timestamp_dt"] = df["timestamp"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S"))



In [3]:
df.sort_values(by="timestamp_dt", ascending=True)

,algorithm,background_maximum_return,background_mean_return,background_minimum_return,centroid_lat,centroid_lon,change_area,compactness,detect_strength,detect_type,...,target_minimum_return,target_total_return,timestamp,weighted_centroid_lat,weighted_centroid_lon,geometry,lattitude,longitude,coordinates,timestamp_dt
4248,ACD-29August2019,3.36,1.27,0.04,31.592990,43.936550,526.00,1.0776,-30.9,Departure,...,36.16,7735.24,2018-11-01T00:00:00,31.592985,43.936581,POINT Z (43.93651 31.59311 0.00000),31.593114,43.936510,"(31.5931135, 43.9365099)",2018-11-01
4249,ACD-29August2019,5.41,2.59,0.00,31.833372,43.656004,282.82,0.9842,-24.7,Departure,...,334.74,3074.55,2018-11-01T00:00:00,31.833377,43.656003,POINT Z (43.65597 31.83350 0.00000),31.833495,43.655968,"(31.8334953, 43.6559675)",2018-11-01
4250,ACD-29August2019,11.84,6.30,1.05,31.879711,43.713683,329.95,1.0846,-23.7,Departure,...,126.76,4628.25,2018-11-01T00:00:00,31.879723,43.713676,POINT Z (43.71365 31.87983 0.00000),31.879835,43.713647,"(31.8798349, 43.7136467)",2018-11-01
4251,ACD-29August2019,7.58,3.16,0.29,32.835074,41.327486,479.16,1.6634,-16.2,Departure,...,51.83,1210.21,2018-11-05T00:00:00,32.835112,41.327497,POINT Z (41.32744 32.83520 0.00000),32.835198,41.327439,"(32.8351977, 41.3274391)",2018-11-05
4252,ACD-29August2019,12.07,4.11,0.00,32.502141,41.416089,359.23,1.0462,-27.2,Departure,...,60.37,968.68,2018-11-05T00:00:00,32.502158,41.416095,POINT Z (41.41604 32.50226 0.00000),32.502264,41.416043,"(32.5022643, 41.4160425)",2018-11-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4218,ACD-29August2019,1.82,0.71,0.23,31.007855,42.221301,422.92,0.9998,40.0,Arrival,...,37.01,33720.83,2019-10-30T00:00:00,31.007813,42.221297,POINT Z (42.22127 31.00798 0.00000),31.007978,42.221265,"(31.0079784, 42.2212653)",2019-10-30
4219,ACD-29August2019,2.28,1.59,0.45,31.022819,42.367975,281.94,1.0310,29.0,Arrival,...,48.38,1773.42,2019-10-30T00:00:00,31.022863,42.367972,POINT Z (42.36794 31.02294 0.00000),31.022943,42.367939,"(31.0229429, 42.367939)",2019-10-30
4220,ACD-29August2019,6.27,1.34,0.28,31.075552,42.526360,328.93,1.0716,27.2,Arrival,...,38.03,2698.03,2019-10-30T00:00:00,31.075552,42.526361,POINT Z (42.52632 31.07568 0.00000),31.075676,42.526325,"(31.0756762, 42.5263246)",2019-10-30
4222,ACD-29August2019,14.63,7.20,0.33,30.996824,42.574917,328.93,1.0846,23.2,Arrival,...,105.10,3462.54,2019-10-30T00:00:00,30.996845,42.574910,POINT Z (42.57488 30.99695 0.00000),30.996948,42.574881,"(30.9969476, 42.5748815)",2019-10-30


## dts in the following cell will be a list of unique datetime objects from sample geojson file##

In [4]:
list_dts = df["timestamp_dt"].to_list()
np_dts = np.array(list_dts)
dts = np.unique(np_dts)

In [5]:
class Collect:
    def __init__(self, coords, time, num):
        self.coords = coords
        self.time = time
        self.num = num
    def print_collect(self):
        print("Date of collect: ", end = " ") 
        print(self.time)
        print("Number of points in collect: ", end = " ") 
        print(self.num)
        print("Coordinates of points: ", end = " ") 
        print(self.coords)
    def get_coords(self):
        return self.coords
    def get_time(self):
        return self.time

In [6]:
class Cluster:
    def __init__(self, coords, time, size):
        self.coords = coords
        self.time = time
        self.size = size
    def new_point(self, coord):
        self.coords.append(coord)
        self.size += 1
    def print_cluster(self):
        print("Date of cluster: ", end = " ")
        print(self.time)
        print("Size of cluster: ", end = " ")
        print(self.size)
        print("Point within collect: ")
        for coord in self.coords:
            print(coord.get_coords())
    def get_coords(self):
        return self.coords
    def add_coords(self, coords):
        self.coords.extend(coords)
    
    # functions used down the line to make new attributes
    def make_centroid(self):
        tot_lat = 0
        tot_lon = 0
        for coord in self.coords:
            c = coord.get_coords()
            tot_lat += c[0]
            tot_lon += c[1]
        lat = tot_lat / len(self.coords)
        lon = tot_lon / len(self.coords)
        self.centroid = (lat, lon)
    def get_centroid(self):
        return self.centroid
    def make_radii(self):
        dist_sum = 0
        dist_max = 0
        for coord in self.coords:
            c = coord.get_coords()
            dist = distance(c, self.centroid).km
            if dist > dist_max:
                dist_max = dist
            dist_sum += dist
        self.max_r = dist_max
        self.avg_r = dist_sum / len(self.coords)
    def get_max_r(self):
        return self.max_r
    def get_avg_r(self):
        return self.avg_r
            
            

In [7]:
# built to keep track of whether a set of coordinates is already in a cluster

class Cluster_Coord:
    def __init__(self, coords, c):
        self.coords = coords
        self.c = c
    def change_status(self, c):
        self.c = c
    def get_status(self):
        return self.c
    def get_coords(self):
        return self.coords

## The following 2 cells can be used to deconstruct the object "master" lists ##

In [8]:
collects = [] # list of collects

In [9]:
clusters = [] # list of clusters

In [10]:
def init_collect(time):
    coords = []
    collect_data = df.loc[df['timestamp_dt']==time, ['coordinates', 'timestamp_dt']]
    num = len(collect_data)
    for i in range(num):
        d = collect_data.iloc[i,:]
        coords.append(d[0])
    collects.append(Collect(coords, time, num))

In [11]:
# initialize all of the collects to the collects array - can be different time period down the line

for time in dts:
    init_collect(time)

idx = 0
for collect in collects:
    print("Index of collects array: " + str(idx))
    idx += 1
    collect.print_collect()
    print()

Index of collects array: 0
Date of collect:  2018-11-01 00:00:00
Number of points in collect:  3
Coordinates of points:  [(31.5931135, 43.9365099), (31.8334953, 43.6559675), (31.8798349, 43.7136467)]

Index of collects array: 1
Date of collect:  2018-11-05 00:00:00
Number of points in collect:  4
Coordinates of points:  [(32.8351977, 41.3274391), (32.5022643, 41.4160425), (32.4978841, 41.4141477), (32.4518448, 41.37964)]

Index of collects array: 2
Date of collect:  2018-11-06 00:00:00
Number of points in collect:  23
Coordinates of points:  [(32.7993297, 42.8858036), (32.3368143, 43.319328), (32.3849988, 43.3193711), (32.384144, 43.3173443), (32.3437482, 43.3284922), (32.343872, 43.3279943), (32.3373096, 43.3204349), (32.3439545, 43.3276754), (31.5390979, 43.4076214), (32.431786, 42.1213852), (32.4875081, 42.1089776), (32.2772893, 42.209378), (32.0276532, 42.3203179), (32.0953926, 42.5449025), (32.0901751, 42.5058812), (32.0380529, 42.2591599), (31.5625085, 42.1659556), (31.5403554, 4

## The following two cells are to show how you can find distances between the points in a collect. This is what we are looking at to form clusters; this is the threshold parameter 

In [12]:
def print_collect_distances(collect):
    coords = collect.get_coords()
    for i in range(len(coords)):
        for j in range(len(coords)):
            if i < j:
                print("Points " + str(i) + " and " + str(j))
                print("First point: " + str(coords[i]))
                print("Second point: " + str(coords[j]))
                dist = distance(coords[i], coords[j]).km
                print("Distance in km between the points: " + str(dist))
                print()

In [13]:
print_collect_distances(collects[12])

Points 0 and 1
First point: (32.9192556, 41.5490473)
Second point: (32.1563655, 41.3386307)
Distance in km between the points: 86.87993811520361

Points 0 and 2
First point: (32.9192556, 41.5490473)
Second point: (32.1630897, 41.3398226)
Distance in km between the points: 86.12823089049672

Points 0 and 3
First point: (32.9192556, 41.5490473)
Second point: (32.1315831, 41.333638)
Distance in km between the points: 89.66345683933014

Points 0 and 4
First point: (32.9192556, 41.5490473)
Second point: (32.1902998, 41.3178343)
Distance in km between the points: 83.70449405098373

Points 0 and 5
First point: (32.9192556, 41.5490473)
Second point: (32.1386244, 41.3350139)
Distance in km between the points: 88.87346364100911

Points 0 and 6
First point: (32.9192556, 41.5490473)
Second point: (32.1966236, 41.3131741)
Distance in km between the points: 83.14279334471857

Points 0 and 7
First point: (32.9192556, 41.5490473)
Second point: (32.1572322, 41.3388944)
Distance in km between the points

First point: (32.1630897, 41.3398226)
Second point: (31.5014716, 41.2451347)
Distance in km between the points: 73.9083384389817

Points 2 and 55
First point: (32.1630897, 41.3398226)
Second point: (31.4212538, 41.3584686)
Distance in km between the points: 82.27608017834481

Points 2 and 56
First point: (32.1630897, 41.3398226)
Second point: (31.4476238, 41.3972168)
Distance in km between the points: 79.51923640366024

Points 2 and 57
First point: (32.1630897, 41.3398226)
Second point: (31.425382, 41.366421)
Distance in km between the points: 81.8381812605984

Points 2 and 58
First point: (32.1630897, 41.3398226)
Second point: (31.5030894, 41.2450988)
Distance in km between the points: 73.73069085977191

Points 2 and 59
First point: (32.1630897, 41.3398226)
Second point: (31.5524237, 41.3822336)
Distance in km between the points: 67.83213955387093

Points 2 and 60
First point: (32.1630897, 41.3398226)
Second point: (31.490803, 41.1705039)
Distance in km between the points: 76.24957336

First point: (32.1572322, 41.3388944)
Second point: (32.0479874, 41.2160094)
Distance in km between the points: 16.77146558016744

Points 7 and 19
First point: (32.1572322, 41.3388944)
Second point: (32.0536237, 41.2217807)
Distance in km between the points: 15.943111768945451

Points 7 and 20
First point: (32.1572322, 41.3388944)
Second point: (32.0551351, 41.2286448)
Distance in km between the points: 15.37709691309324

Points 7 and 21
First point: (32.1572322, 41.3388944)
Second point: (32.0892252, 41.2849583)
Distance in km between the points: 9.098119247943442

Points 7 and 22
First point: (32.1572322, 41.3388944)
Second point: (31.9896791, 41.4721793)
Distance in km between the points: 22.44039860757112

Points 7 and 23
First point: (32.1572322, 41.3388944)
Second point: (32.0035222, 41.3551926)
Distance in km between the points: 17.11394183695226

Points 7 and 24
First point: (32.1572322, 41.3388944)
Second point: (31.9641031, 41.2193295)
Distance in km between the points: 24.20

Second point: (32.1913729, 41.047383)
Distance in km between the points: 27.75314992960683

Points 10 and 18
First point: (32.1580233, 41.3390237)
Second point: (32.0479874, 41.2160094)
Distance in km between the points: 16.84331459325276

Points 10 and 19
First point: (32.1580233, 41.3390237)
Second point: (32.0536237, 41.2217807)
Distance in km between the points: 16.014841243877036

Points 10 and 20
First point: (32.1580233, 41.3390237)
Second point: (32.0551351, 41.2286448)
Distance in km between the points: 15.449994658577179

Points 10 and 21
First point: (32.1580233, 41.3390237)
Second point: (32.0892252, 41.2849583)
Distance in km between the points: 9.177728707980805

Points 10 and 22
First point: (32.1580233, 41.3390237)
Second point: (31.9896791, 41.4721793)
Distance in km between the points: 22.50623231458075

Points 10 and 23
First point: (32.1580233, 41.3390237)
Second point: (32.0035222, 41.3551926)
Distance in km between the points: 17.20022481275455

Points 10 and 24
F

First point: (32.2831667, 41.1336529)
Second point: (31.8240731, 41.2481632)
Distance in km between the points: 52.043770480813336

Points 14 and 45
First point: (32.2831667, 41.1336529)
Second point: (31.8032895, 41.3255532)
Distance in km between the points: 56.21444220130634

Points 14 and 46
First point: (32.2831667, 41.1336529)
Second point: (31.92784, 41.0687621)
Distance in km between the points: 39.87487675881766

Points 14 and 47
First point: (32.2831667, 41.1336529)
Second point: (31.9272277, 41.0598701)
Distance in km between the points: 40.07924590662279

Points 14 and 48
First point: (32.2831667, 41.1336529)
Second point: (31.96799, 41.1776064)
Distance in km between the points: 35.194879983325194

Points 14 and 49
First point: (32.2831667, 41.1336529)
Second point: (31.8725663, 41.0102427)
Distance in km between the points: 46.997921858450255

Points 14 and 50
First point: (32.2831667, 41.1336529)
Second point: (31.9814881, 40.9383792)
Distance in km between the points: 3

...

Points 23 and 49
First point: (32.0035222, 41.3551926)
Second point: (31.8725663, 41.0102427)
Distance in km between the points: 35.70363840273701

Points 23 and 50
First point: (32.0035222, 41.3551926)
Second point: (31.9814881, 40.9383792)
Distance in km between the points: 39.46489143653559

Points 23 and 51
First point: (32.0035222, 41.3551926)
Second point: (31.7873666, 40.9888566)
Distance in km between the points: 42.13646432184488

Points 23 and 52
First point: (32.0035222, 41.3551926)
Second point: (31.8241789, 40.9897173)
Distance in km between the points: 39.87931415286549

Points 23 and 53
First point: (32.0035222, 41.3551926)
Second point: (31.7447209, 40.9481244)
Distance in km between the points: 48.03250662519667

Points 23 and 54
First point: (32.0035222, 41.3551926)
Second point: (31.5014716, 41.2451347)
Distance in km between the points: 56.636805856754115

Points 23 and 55
First point: (32.0035222, 41.3551926)
Second point: (31.4212538, 41.3584686)
Distance in km be

Distance in km between the points: 0.8177687067154629

Points 27 and 32
First point: (32.0666005, 41.2530766)
Second point: (31.9837683, 41.2812815)
Distance in km between the points: 9.563686969935027

Points 27 and 33
First point: (32.0666005, 41.2530766)
Second point: (32.0098756, 41.502642)
Distance in km between the points: 24.397214344738803

Points 27 and 34
First point: (32.0666005, 41.2530766)
Second point: (32.0143688, 41.1179784)
Distance in km between the points: 14.013177880823598

Points 27 and 35
First point: (32.0666005, 41.2530766)
Second point: (31.9915902, 41.3194276)
Distance in km between the points: 10.414819130407247

Points 27 and 36
First point: (32.0666005, 41.2530766)
Second point: (32.0424922, 41.5164498)
Distance in km between the points: 25.01546398371192

Points 27 and 37
First point: (32.0666005, 41.2530766)
Second point: (31.9269697, 41.1391799)
Distance in km between the points: 18.856513458125416

Points 27 and 38
First point: (32.0666005, 41.2530766)

Distance in km between the points: 17.358503241645376

Points 34 and 51
First point: (32.0143688, 41.1179784)
Second point: (31.7873666, 40.9888566)
Distance in km between the points: 27.978101789609088

Points 34 and 52
First point: (32.0143688, 41.1179784)
Second point: (31.8241789, 40.9897173)
Distance in km between the points: 24.32908073424459

Points 34 and 53
First point: (32.0143688, 41.1179784)
Second point: (31.7447209, 40.9481244)
Distance in km between the points: 33.945179251440976

Points 34 and 54
First point: (32.0143688, 41.1179784)
Second point: (31.5014716, 41.2451347)
Distance in km between the points: 58.133270876643955

Points 34 and 55
First point: (32.0143688, 41.1179784)
Second point: (31.4212538, 41.3584686)
Distance in km between the points: 69.60377061337192

Points 34 and 56
First point: (32.0143688, 41.1179784)
Second point: (31.4476238, 41.3972168)
Distance in km between the points: 68.1863098583519

Points 34 and 57
First point: (32.0143688, 41.1179784)


Points 40 and 47
First point: (31.8049612, 41.3256575)
Second point: (31.9272277, 41.0598701)
Distance in km between the points: 28.572760706949076

Points 40 and 48
First point: (31.8049612, 41.3256575)
Second point: (31.96799, 41.1776064)
Distance in km between the points: 22.868963657582164

Points 40 and 49
First point: (31.8049612, 41.3256575)
Second point: (31.8725663, 41.0102427)
Distance in km between the points: 30.783258991943693

Points 40 and 50
First point: (31.8049612, 41.3256575)
Second point: (31.9814881, 40.9383792)
Distance in km between the points: 41.538538907937074

Points 40 and 51
First point: (31.8049612, 41.3256575)
Second point: (31.7873666, 40.9888566)
Distance in km between the points: 31.95517913853677

Points 40 and 52
First point: (31.8049612, 41.3256575)
Second point: (31.8241789, 40.9897173)
Distance in km between the points: 31.87905166501862

Points 40 and 53
First point: (31.8049612, 41.3256575)
Second point: (31.7447209, 40.9481244)
Distance in km b

Distance in km between the points: 53.867663583222175

Points 53 and 57
First point: (31.7447209, 40.9481244)
Second point: (31.425382, 41.366421)
Distance in km between the points: 53.19836401514071

Points 53 and 58
First point: (31.7447209, 40.9481244)
Second point: (31.5030894, 41.2450988)
Distance in km between the points: 38.88062545017392

Points 53 and 59
First point: (31.7447209, 40.9481244)
Second point: (31.5524237, 41.3822336)
Distance in km between the points: 46.369054872899454

Points 53 and 60
First point: (31.7447209, 40.9481244)
Second point: (31.490803, 41.1705039)
Distance in km between the points: 35.18356734045814

Points 53 and 61
First point: (31.7447209, 40.9481244)
Second point: (31.5014539, 41.2448117)
Distance in km between the points: 38.986312475079274

Points 53 and 62
First point: (31.7447209, 40.9481244)
Second point: (31.4176961, 41.3035018)
Distance in km between the points: 49.52458380206654

Points 53 and 63
First point: (31.7447209, 40.9481244)
Sec

In [14]:
def find_cluster(cc): #pass in 2 cluster_coords
    for cluster in clusters:
        for point in cluster.get_coords():
            if point == cc:
                return cluster
    return 0

In [15]:
def form_clusters_by_collect(collect, km_dist):
    cc = []
    coords = collect.get_coords()
    #init Cluster_Coords
    for coord in coords:
        cc.append(Cluster_Coord(coord, False))
    for i in range(len(cc)):
        for j in range(len(cc)):
            if (j > i):
                c1 = cc[i].get_coords()
                c2 = cc[j].get_coords()
                dist = distance(c1, c2).km
                if dist <= km_dist:
                    if cc[i].get_status() == True and cc[j].get_status() == True:
                        # check if in same cluster
                        clust1 = find_cluster(cc[i])
                        clust2 = find_cluster(cc[j])
                        if clust1 != clust2:
                            # concatenate clusters
                            idx1 = -1
                            idx1 = clusters.index(clust1)
                            idx2 = -1
                            idx2 = clusters.index(clust2)
                            if idx != -1:
                                c_list = clusters[idx2].get_coords()
                                clusters[idx1].add_coords(c_list)
                                clusters.remove(clust2)
                        # else do nothing - already in a cluster together
                        
                    elif cc[i].get_status() == True and cc[j].get_status() == False:
                        clust = find_cluster(cc[i])
                        idx = -1
                        idx = clusters.index(clust)
                        clusters[idx].new_point(cc[j])
                        cc[j].change_status(True)
                        
                    elif cc[i].get_status() == False and cc[j].get_status() == True:
                        clust = find_cluster(cc[j])
                        idx = -1
                        idx = clusters.index(clust)
                        clusters[idx].new_point(cc[i])
                        cc[i].change_status(True)
                        
                    else: 
                        # create new cluster (size of 2 because every cluster starts with 2 points)
                        clusters.append(Cluster([cc[i], cc[j]], collect.get_time(), 2))
                        cc[i].change_status(True)
                        cc[j].change_status(True)


In [16]:
clusters = []

form_clusters_by_collect(collects[12], 5)

idx = 0
for cluster in clusters:
    print("Index in clusters array: " + str(idx))
    idx += 1
    cluster.print_cluster()
    print()

Index in clusters array: 0
Date of cluster:  2018-11-23 00:00:00
Size of cluster:  12
Point within collect: 
(32.1563655, 41.3386307)
(32.1630897, 41.3398226)
(32.1315831, 41.333638)
(32.1902998, 41.3178343)
(32.1386244, 41.3350139)
(32.1572322, 41.3388944)
(32.1580233, 41.3390237)
(32.1451133, 41.3423322)
(32.1966236, 41.3131741)
(32.1985546, 41.311203)
(32.2075111, 41.3088607)
(32.2498791, 41.3151301)

Index in clusters array: 1
Date of cluster:  2018-11-23 00:00:00
Size of cluster:  3
Point within collect: 
(32.3024035, 41.1437899)
(32.2769668, 41.1330734)
(32.2831667, 41.1336529)

Index in clusters array: 2
Date of cluster:  2018-11-23 00:00:00
Size of cluster:  3
Point within collect: 
(32.0892252, 41.2849583)
(32.0977735, 41.2897808)
(32.0479874, 41.2160094)
(32.0536237, 41.2217807)
(32.0551351, 41.2286448)
(32.0666005, 41.2530766)
(32.0723739, 41.2584653)
(32.0908391, 41.2844475)

Index in clusters array: 3
Date of cluster:  2018-11-23 00:00:00
Size of cluster:  4
Point within c

## As with collects, the next two cells are some code that can actually tell us distances between the points in a specific cluster. Hopefully, this can help us make informed decisions about the effectiveness of a chosen threshold. Remember that distances are in km.

In [17]:
def print_cluster_distances(cluster):
    coords = cluster.get_coords()
    for i in range(len(coords)):
        for j in range(len(coords)):
            if i < j:
                print("Points " + str(i) + " and " + str(j))
                print("First point: " + str(coords[i].get_coords()))
                print("Second point: " + str(coords[j].get_coords()))
                dist = distance(coords[i].get_coords(), coords[j].get_coords()).km
                print("Distance in km between the points: " + str(dist))
                print()

In [18]:
idx = 0
for cluster in clusters:
    print("INDEX OF CLUSTER IN ARRAY: " + str(idx))
    print_cluster_distances(cluster)
    print()
    print()

INDEX OF CLUSTER IN ARRAY: 0
Points 0 and 1
First point: (32.1563655, 41.3386307)
Second point: (32.1630897, 41.3398226)
Distance in km between the points: 0.7540726049904244

Points 0 and 2
First point: (32.1563655, 41.3386307)
Second point: (32.1315831, 41.333638)
Distance in km between the points: 2.788180492015556

Points 0 and 3
First point: (32.1563655, 41.3386307)
Second point: (32.1902998, 41.3178343)
Distance in km between the points: 4.243474085107758

Points 0 and 4
First point: (32.1563655, 41.3386307)
Second point: (32.1386244, 41.3350139)
Distance in km between the points: 1.996671223909669

Points 0 and 5
First point: (32.1563655, 41.3386307)
Second point: (32.1572322, 41.3388944)
Distance in km between the points: 0.0992750035799419

Points 0 and 6
First point: (32.1563655, 41.3386307)
Second point: (32.1580233, 41.3390237)
Distance in km between the points: 0.18753352943138157

Points 0 and 7
First point: (32.1563655, 41.3386307)
Second point: (32.1451133, 41.3423322)


## Now let's have some more fun :-)
## I am now going to start making new cluster attributes in order to make visual representations of clusters.

In [19]:
for cluster in clusters:
    cluster.make_centroid()
    cluster.make_radii()
    print(cluster.get_centroid())
    print(cluster.get_max_r())
    print(cluster.get_avg_r())
    print()
    


(32.174408308333334, 41.32779647499999)
8.453757731052846
3.4170612723636293

(32.28751233333333, 41.136838733333335)
1.7763711310275538
1.1886986627123755

(32.0716948, 41.254645425)
4.496912824320006
2.949276227273169

(32.012732975, 41.498466650000005)
3.7115101073371073
2.090009216541166

(31.99296023333333, 41.3186339)
3.6741984127199494
2.497121689914865

(31.966457199999997, 41.19155933333334)
2.638025454179042
1.7590438381966154

(32.02797795, 41.1233831)
1.5931180352356813
1.5931074741348898

(31.9815677, 40.9536707)
1.4452584450341992
1.4452578212030063

(31.911284728571424, 41.02834278571429)
4.622167874138937
2.592142917714457

(31.80412535, 41.325605350000004)
0.09281335106928049
0.09281334616783973

(31.80577275, 40.98928695)
2.0413464497220373
2.041343576605972

(31.502004966666664, 41.24501506666667)
0.12050279118036901
0.08160219982228484

(31.42198467142857, 41.33997908571428)
6.139316750033687
3.2028155166893697



In [20]:
import folium

In [21]:
m = folium.Map(location=[31.634036, 42.556084], zoom_start=8)
for cluster in clusters:
    r = cluster.get_max_r() * 1000
    c = cluster.get_centroid()
    folium.Circle(
        location = c,
        radius = r,
        color = 'crimson',
        fill = True,
        fill_color = 'crimson'
    ).add_to(m)

In [22]:
# save the map as an html

m.save('test3_map.html')